<a href="https://colab.research.google.com/github/MengOonLee/AccountReceivable/blob/main/Workflow/InvoicePayment/FineTune_Llama2/Forecast_DaysToSettle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install --no-cache-dir -qU \
    datasets transformers bitsandbytes accelerate peft trl

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import datasets

dataset = datasets.load_dataset("mlabonne/guanaco-llama2-1k", split='train')
print(dataset[500])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': '<s>[INST] What is the link between race and intelligence? [/INST] There is no scientific evidence to support a link between race and intelligence. Intelligence is a complex trait that is influenced by many factors, including genetics, environment, education, and culture. Race, on the other hand, is a social construct that is based on physical characteristics such as skin color, hair texture, and facial features. While there may be genetic differences between individuals of different races, these differences do not correspond to any meaningful differences in intelligence.\n\nIt is important to recognize that any attempts to link race and intelligence can have harmful and divisive effects on society. The notion that certain races are inherently more or less intelligent than others has been used throughout history to justify discrimination and oppression, and it has no scientific basis. It is essential to base our understanding of intelligence on rigorous scientific research, fr

In [4]:
import torch
import transformers

base_model = "meta-llama/Llama-2-7b-chat-hf"

quantization_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quantization_config,
    device_map='auto'
)
model.config.use_cache = False

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
import transformers
import peft
import trl

training_args = transformers.TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_steps=15,
    warmup_steps=2
)

peft_config = peft.LoraConfig(
    lora_alpha=64, lora_dropout=0.1, r=16,
    bias='none', task_type='CAUSAL_LM'
)

trainer = trl.SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

trainer.train()

peft_model = "peft_model"
trainer.model.save_pretrained(peft_model)

del model
del trainer
torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:173: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.462300
2,2.452700
3,2.005700
4,1.799800
5,1.672000
6,2.472400
7,1.781000
8,1.522200
9,1.864800
10,2.060800


In [9]:
import torch
import transformers
import peft

pretrained_model = transformers.AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
finetuned_model = peft.PeftModel.from_pretrained(
    pretrained_model, peft_model
)
finetuned_model = finetuned_model.merge_and_unload()
finetuned_dir = "finetuned_Llama2_forecast_DaysToSettle"
finetuned_model.save_pretrained(finetuned_dir)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(finetuned_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


('finetuned_Llama2_forecast_DaysToSettle/tokenizer_config.json',
 'finetuned_Llama2_forecast_DaysToSettle/special_tokens_map.json',
 'finetuned_Llama2_forecast_DaysToSettle/tokenizer.model',
 'finetuned_Llama2_forecast_DaysToSettle/added_tokens.json',
 'finetuned_Llama2_forecast_DaysToSettle/tokenizer.json')

In [11]:
import transformers

prompt = "What is a large language model?"
pipe = transformers.pipeline(
    task="text-generation", 
    model=finetuned_model,
    tokenizer=tokenizer)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is a large language model? [/INST]  A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. The model can be used for a wide range of natural language processing (NLP) tasks, such as text generation, language translation, and text summarization.

The key advantage of LLMs is their ability to generate text that is coherent and natural-sounding, even when the input is ambiguous or unclear. This is because the model has been trained on a large dataset of text and can draw on this knowledge to generate coherent and fluent output.

There are several different types of LLMs, including:

1. Recurrent neural networks (RNNs): These are the most common type of LLM and use a loop of recurrent connections to generate text one word or one character at a time.
2. Transformers: These are a type of deep learning model that use self-attention mechanisms to 